In [1]:
import hmmlearn as hm


In [2]:
import numpy as np
from hmmlearn import hmm
np.random.seed(42)

model = hmm.GaussianHMM(n_components=3, covariance_type="full")
model.startprob_ = np.array([0.6, 0.3, 0.1])
model.transmat_ = np.array([[0.7, 0.2, 0.1],
                          [0.3, 0.5, 0.2],
                         [0.3, 0.3, 0.4]])
model.means_ = np.array([[0.0, 0.0], [3.0, -3.0], [5.0, 10.0]])
model.covars_ = np.tile(np.identity(2), (3, 1, 1))
X, Z = model.sample(100)

In [8]:
#Parameters
n_1 = 1500 #Individuals of ancestry one. TODO: Find real number
n_2 = 1500 #Individuals of ancestry two. TODO: Find real number
n = [n_1, n_2]
T = 10 #?????????????? Wild guess
mu_1 = n_1 / (n_1 + n_2) #????????????? Wild guess
mu_2 = 1 - mu_1
mu = [mu_1, mu_2]
p_1 = 0.05
p_2 = 0.05
p = [p_1, p_2]
rho_1 = 60000 / n_1
rho_2 = 90000 / n_2
rho = [rho_1, rho_2]
theta_1 = 0.2/(0.2+n_1)
theta_2 = 0.2/(0.2+n_2)
theta_3 = 0.01
theta = [theta_1, theta_2]
def transition_prob(i, j, k, l, m, n, r):
    if(i != l and m == l):
        return (1 - np.exp(-r*T)) * mu[l] * (1 - p[l])/n[m]
    elif(i != l and m != l):
        return (1 - np.exp(-r*rho[l])) * mu[l] * p[l]/n[m]
    elif(i == l and m == l and (j != m or k != n)):
        return np.exp(-r*T)*(1 - np.exp(-r*rho[l])) * (1 - p[l])/n[m] 
        + (1 - np.exp(-r*T)) * mu[l] * (1 - p[l])/n[m]
    elif(i == l and m == l and j == m and k == n):
        return np.exp(-r*T)*np.exp(-r*rho[l])
        + np.exp(-r*T)*(1 - np.exp(-r*rho[l]))*(1 - p[l])/n[m] 
        + (1 - np.exp(-r*T)) * mu[l] * (1 - p[l])/n[m]
    elif(i == l and m != l and (j != m or k != n)):
        return np.exp(-r*T)*(1 - np.exp(-r*rho[l])) * p[l]/n[m] 
        + (1 - np.exp(-r*T)) * mu[l] * p[l] / n[m] 
    elif(i == l and m != l and j == m and k == n):
        return np.exp(-r*T)*np.exp(-r*rho[l])
        + np.exp(-r*T) * (1 - np.exp(-r*rho[l])) * p[l]/n[m]
        + (1 - np.exp(-r*T)) * mu[l] * p[l]/n[m]
        
def emmission_prob():
    pass